<h1>Start MongoDB</h1>

To launch the MongoDB server, run <tt>mongod</tt>. (Note: there is no "b" at the end.) Use the optional <tt>dbpath</tt> argument to specify the path for storing files:<br/>
<center>
<tt>mongod --dbpath=/Users/ramcdougal/mongodbdata</tt>
</center><br/>
Note: run this command from the console; do not run it from inside Python.<br/><br/>
On Windows, the default path is <tt>"C:\Program Files\MongoDB\Server\3.4\bin\mongod.exe"</tt>
<br/><br/>
Basic information on your local MongoDB server is available at <a href="http://localhost:28017/">http://localhost:28017/</a> (The port number here is 1000 more than the port for programmatically connecting to the database endpoint.) To enable a richer RESTful web interface add <tt>--rest</tt> to the command to launch MongoDB. Note that if this port is not firewalled, this is a potential security vulnerability.

<h1>Connect to MongoDB</h1>

In [1]:
from pymongo import MongoClient
mongodb = MongoClient()         # can pass in a mongodb:// URL to connect to a server on a different machine

<h1>Create your first database</h1>

In [2]:
test = mongodb.test

Alternatively, to create a database with a name that would be invalid Python (e.g. beginning with a number), you can use:

In [3]:
test = mongodb['test']

If you need to authenticate (and in a production environment, your databases should be password protected at least for writing), now is the time:

In [ ]:
test.authenticate('username', 'password')

An invalid authentication attemp will raise a <tt>pymongo.errors.OperationFailure</tt> exception.

If for some reason, you want to drop (delete) an entire database, use:

In [5]:
mongodb.drop_database('test')

<h1>Create a collection</h1>

Like a database, a collection can be created inside a database by simply referring to its name:

In [6]:
sciencedb = mongodb.sciencedb # this creates a new database called sciencedb

In [34]:
cells = sciencedb.cells # or sciencedb['cells']

Since mongodb is schemaless, there is no more information to give.

<h1>Adding data to a collection</h1>

Use <tt>insert_one</tt> or <tt>insert_many</tt> to insert one or many documents into a collection (older versions of pymongo used <tt>insert</tt> for both of these, but this is deprecated):

In [35]:
cells.insert_one({
                  'name': 'Hippocampus CA1 pyramidal cell',
                  'channels': ['I Na,t', 'I Na,p', 'I Potassium', 'I A', 'I K', 'I M',
                               'I L high threshold', 'I N', 'I T low threshold', 'I p,q',
                               'I K,Ca', 'I h'],
                  'transmitters': ['NO', 'Glutamate'],
                  'average dendrite length': 4586
              })

In [36]:
insert_result = cells.insert_many([
                 {
                      'name': 'Dentate Gyrus granule cell',
                      'average dendrite length': 3500,
                      'excitatory': True,
                      'channels': ['I Na,t', 'I Na,p', 'I A', 'I K', 'I L high threshold',
                                   'I N', 'I T low threshold', 'I K,Ca', 'I h'],
                      'transmitters': ['Dynorphin', 'Zn2+', 'Glutamate'],
                 },
                 {
                      'name': 'Olfactory granule cell',
                      'average dendrite length': 1000,   # I made this up completely, please don't quote any of these
                      'excitatory': False,
                      'channels': ['I A', 'I N', 'I T low threshold', 'I p,q', 'I CAN', 'I h'],
                      'transmitters': ['NO', 'Gaba'],
                      'region': 'olfactory bulb'
                 }
              ])

In [37]:
insert_result.inserted_ids

[ObjectId('589352986c132956d638d760'), ObjectId('589352986c132956d638d761')]

Note that <tt>insert_one</tt> and <tt>insert_many</tt> return an object which contains the id of the newly created document(s). An <tt>id</tt> may also be specified by including an <tt>_id</tt> key in the dictionary. (Note the underscore.)<br/><br/>We have thus now defined a total of <tt>cells.count()</tt> documents (cells).

In [38]:
cells.count()

3

<h1>Indexes and uniqueness</h1>

To speed up searches, we can define an index:

In [12]:
cells.create_index('name', unique=True)

u'name_1'

The <tt>unique=True</tt> here ensures that there is only one cell in the database with a given name. Attempting to add another e.g. Hippocampus CA1 pyramidal cell will raise a <tt>pymongo.errors.DuplicateKeyError</tt> Exception:

In [13]:
cells.insert_one({
                  'name': 'Hippocampus CA1 pyramidal cell',
                  'average dendrite length': 4586
              })

DuplicateKeyError: E11000 duplicate key error collection: sciencedb.cells index: name_1 dup key: { : "Hippocampus CA1 pyramidal cell" }

<h1>Querying</h1>

In [14]:
from pprint import pprint
for doc in cells.find({'name': 'Hippocampus CA1 pyramidal cell'}):
    pprint(doc)

{u'_id': ObjectId('58934f516c132956d638d759'),
 u'average dendrite length': 4586,
 u'channels': [u'I Na,t',
               u'I Na,p',
               u'I Potassium',
               u'I A',
               u'I K',
               u'I M',
               u'I L high threshold',
               u'I N',
               u'I T low threshold',
               u'I p,q',
               u'I K,Ca',
               u'I h'],
 u'name': u'Hippocampus CA1 pyramidal cell',
 u'transmitters': [u'NO', u'Glutamate']}


If we know there is only one match (or if all we need is one), we can use <tt>find_one</tt> instead and then no iteration is necessary:

In [15]:
pprint(cells.find_one({'transmitters': 'Gaba'}))

{u'_id': ObjectId('58934f516c132956d638d75b'),
 u'average dendrite length': 1000,
 u'channels': [u'I A',
               u'I N',
               u'I T low threshold',
               u'I p,q',
               u'I CAN',
               u'I h'],
 u'excitatory': False,
 u'name': u'Olfactory granule cell',
 u'region': u'olfactory bulb',
 u'transmitters': [u'NO', u'Gaba']}


Note that the search found Gaba as one of multiple transmitters. We can perform more complicated queries using <tt>\$and</tt>, <tt>\$or</tt>, <tt>\$not</tt>, <tt>\$gt</tt>, <tt>\$in</tt>, <tt>\$regex</tt> and other <a href="https://docs.mongodb.com/manual/reference/operator/query/">operators</a>.

In [16]:
for doc in cells.find({'$and': [{'channels': 'I A'}, {'channels': 'I L high threshold'}]}):
    pprint(doc)

{u'_id': ObjectId('58934f516c132956d638d759'),
 u'average dendrite length': 4586,
 u'channels': [u'I Na,t',
               u'I Na,p',
               u'I Potassium',
               u'I A',
               u'I K',
               u'I M',
               u'I L high threshold',
               u'I N',
               u'I T low threshold',
               u'I p,q',
               u'I K,Ca',
               u'I h'],
 u'name': u'Hippocampus CA1 pyramidal cell',
 u'transmitters': [u'NO', u'Glutamate']}
{u'_id': ObjectId('58934f516c132956d638d75a'),
 u'average dendrite length': 3500,
 u'channels': [u'I Na,t',
               u'I Na,p',
               u'I A',
               u'I K',
               u'I L high threshold',
               u'I N',
               u'I T low threshold',
               u'I K,Ca',
               u'I h'],
 u'excitatory': True,
 u'name': u'Dentate Gyrus granule cell',
 u'transmitters': [u'Dynorphin', u'Zn2+', u'Glutamate']}


<h3>Example of searching for numeric matches in interval:</h3>

In [17]:
for doc in cells.find({'average dendrite length': {'$lt': 2000}}):
    pprint(doc)

{u'_id': ObjectId('58934f516c132956d638d75b'),
 u'average dendrite length': 1000,
 u'channels': [u'I A',
               u'I N',
               u'I T low threshold',
               u'I p,q',
               u'I CAN',
               u'I h'],
 u'excitatory': False,
 u'name': u'Olfactory granule cell',
 u'region': u'olfactory bulb',
 u'transmitters': [u'NO', u'Gaba']}


<h3>Getting distinct results</h3>

To gather the distinct entries of a field from any search results, use <tt>.distinct</tt>

In [18]:
cells.find().distinct('channels')

[u'I A',
 u'I K',
 u'I K,Ca',
 u'I L high threshold',
 u'I M',
 u'I N',
 u'I Na,p',
 u'I Na,t',
 u'I Potassium',
 u'I T low threshold',
 u'I h',
 u'I p,q',
 u'I CAN']

<h3>Sorting results</h3>

We can sort the results of a search based on a field or fields. We use 1 to indicate ascending order, -1 to indicate descending order.

In [19]:
for cell in cells.find().sort([('average dendrite length', 1)]):
    print '{:35s} {:5d}'.format(cell['name'], cell['average dendrite length'])

Olfactory granule cell               1000
Dentate Gyrus granule cell           3500
Hippocampus CA1 pyramidal cell       4586


In [20]:
for cell in cells.find().sort([('average dendrite length', -1)]):
    print '{:35s} {:5d}'.format(cell['name'], cell['average dendrite length'])

Hippocampus CA1 pyramidal cell       4586
Dentate Gyrus granule cell           3500
Olfactory granule cell               1000


Part of the reason that this works is that the queries don't return a list; they return a <b>cursor</b> object which allows extracting part of the results.

In [21]:
type(cells.find())

pymongo.cursor.Cursor

This doesn't stop you, however from grabbing a specific item or slice. This is most useful, for example, if in the above we only wanted to receive (from the database) the 2 cells with the longest combined dendritic length. Unlike if a list was sent, using a cursor means no additional data would be sent, so we don't have to deal with bandwidth or sorting it on our local machine. We can also use this to <b>page</b> the results.

In [22]:
for cell in cells.find().sort([('average dendrite length', -1)])[:2]:
    print cell['name']

Hippocampus CA1 pyramidal cell
Dentate Gyrus granule cell


<h1>Update</h1>

<h3>Basic use</h3>

Let's declare a region for the cells in the hippocampus, which here we'll select with a regular expression:

In [23]:
cells.update_many({'name': {'$regex': '(Hippocampus|Dentate)'}},
                  {'$set': {'region': 'hippocampus'}})

There is also an <tt>upsert</tt> command that updates entries if it finds a matching one or inserts a new entry if no matching one already exists.

<h3>Numeric updates</h3> 

Suppose for some reason we wanted to convert all dendritic lengths from microns to mm; that is we must multiply them by 0.001. Fortunately, MongoDB provides <a href="https://docs.mongodb.com/manual/reference/operator/update-field/">quantitative</a> update operators such as \$inc and \$mul. 

In [25]:
cells.update_many({},
                 {'$mul': {'average dendrite length': 0.001}})

Let's see the results:

In [26]:
for cell in cells.find():
    print '{:35s} {:8g} mm'.format(cell['name'], cell['average dendrite length'])

Hippocampus CA1 pyramidal cell         4.586 mm
Dentate Gyrus granule cell               3.5 mm
Olfactory granule cell                     1 mm


<h3>Updating within an array</h3>

If you use <tt>\$set</tt> a field, you'll replace the entire field. To replace a matched field within an array, use <tt>.\$</tt>

In [50]:
cells.update_many({'transmitters': 'Gaba'},
                  {'$set': {'transmitters.$': 'GABA'}}
)

Let's check that (1) Gaba has been replaced by GABA and (2) transmitters is still an array.

In [59]:
for cell in cells.find():
    print '{:40} {:30}'.format(cell['name'], cell['transmitters'])

Hippocampus CA1 pyramidal cell           [u'NO', u'Glutamate']         
Dentate Gyrus granule cell               [u'Dynorphin', u'Zn2+', u'Glutamate']
Olfactory granule cell                   [u'NO', u'GABA']              


It worked (see the entry for "Olfactory granule cell" above). Although GABA appears only once in our test dataset, it is a common neurotransmitter.

<h1>Joining documents</h1>

Let's add a new collection:

In [27]:
sciencedb.functions.insert_many([
    {
        'name': 'olfactory bulb',
        'function': 'olfaction'
    },
    {
        'name': 'hippocampus',
        'function': 'learning and memory'
    }])

In [28]:
for cell in cells.aggregate([
        {
            '$lookup':
            {
                'from': 'functions',
                'localField': 'region',
                'foreignField': 'name',
                'as': 'cellFunctions'
            }
        }
    ]):
    pprint(cell)
    break

{u'_id': ObjectId('58934f516c132956d638d759'),
 u'average dendrite length': 4.586,
 u'cellFunctions': [{u'_id': ObjectId('589352656c132956d638d75e'),
                     u'function': u'learning and memory',
                     u'name': u'hippocampus'}],
 u'channels': [u'I Na,t',
               u'I Na,p',
               u'I Potassium',
               u'I A',
               u'I K',
               u'I M',
               u'I L high threshold',
               u'I N',
               u'I T low threshold',
               u'I p,q',
               u'I K,Ca',
               u'I h'],
 u'name': u'Hippocampus CA1 pyramidal cell',
 u'region': u'hippocampus',
 u'transmitters': [u'NO', u'Glutamate']}


So what happened here is that the entire function document got embedded into the cell. Sometimes this is what you want. Othertimes you may want to extract just a certain piece of information. To do this use a <b>projection</b>, which will allow you to drop, include, add, or rename fields:

In [29]:
for cell in cells.aggregate([
        {
            '$lookup':
            {
                'from': 'functions',
                'localField': 'region',
                'foreignField': 'name',
                'as': 'cellFunctions'
            }
        },
        {
            '$project':
            {
                '_id': False,
                'name': True,
                'functions': '$cellFunctions.function'
            }
        }
    ]):
    pprint(cell)


{u'functions': [u'learning and memory'],
 u'name': u'Hippocampus CA1 pyramidal cell'}
{u'functions': [u'learning and memory'],
 u'name': u'Dentate Gyrus granule cell'}
{u'functions': [u'olfaction'], u'name': u'Olfactory granule cell'}


<h1>Aggregation</h1>

Joining is a special case of MongoDB's aggregation pipeline which allows you to combine documents, do certain calculations, group data (\$group), etc. For more, see the <a href="http://api.mongodb.com/python/current/examples/aggregation.html">PyMongo aggregation</a> documentation.

Continuing the previous example, suppose I only wanted to know the functions for cells in the hippocampus. In that case, I'll begin by doing a <tt>\$match</tt> to find just those cells before doing the <tt>\$lookup</tt>. The arguments to <tt>\$match</tt> are the same as you would send to <tt>find_many</tt>.

In [30]:
for cell in cells.aggregate([
        {
            '$match':
            {
                'region': 'hippocampus'
            }
        },
        {
            '$lookup':
            {
                'from': 'functions',
                'localField': 'region',
                'foreignField': 'name',
                'as': 'cellFunctions'
            }
        },
        {
            '$project':
            {
                '_id': False,
                'name': True,
                'functions': '$cellFunctions.function'
            }
        }
    ]):
    pprint(cell)


{u'functions': [u'learning and memory'],
 u'name': u'Hippocampus CA1 pyramidal cell'}
{u'functions': [u'learning and memory'],
 u'name': u'Dentate Gyrus granule cell'}


Suppose we want to get a count of the number of times a given channel appears. We begin by <tt>\$unwind</tt>ing the channels array. This creates one virtual document for each channel. (If we didn't do this, we'd count the number of times a particular set of channels appears. This is interesting information, but not what we want here.)

In [51]:
list(cells.aggregate([
        {
            '$unwind': '$channels'
        }
    ]))

[{u'_id': ObjectId('589352976c132956d638d75f'),
  u'average dendrite length': 4586,
  u'channels': u'I Na,t',
  u'name': u'Hippocampus CA1 pyramidal cell',
  u'transmitters': [u'NO', u'Glutamate']},
 {u'_id': ObjectId('589352976c132956d638d75f'),
  u'average dendrite length': 4586,
  u'channels': u'I Na,p',
  u'name': u'Hippocampus CA1 pyramidal cell',
  u'transmitters': [u'NO', u'Glutamate']},
 {u'_id': ObjectId('589352976c132956d638d75f'),
  u'average dendrite length': 4586,
  u'channels': u'I Potassium',
  u'name': u'Hippocampus CA1 pyramidal cell',
  u'transmitters': [u'NO', u'Glutamate']},
 {u'_id': ObjectId('589352976c132956d638d75f'),
  u'average dendrite length': 4586,
  u'channels': u'I A',
  u'name': u'Hippocampus CA1 pyramidal cell',
  u'transmitters': [u'NO', u'Glutamate']},
 {u'_id': ObjectId('589352976c132956d638d75f'),
  u'average dendrite length': 4586,
  u'channels': u'I K',
  u'name': u'Hippocampus CA1 pyramidal cell',
  u'transmitters': [u'NO', u'Glutamate']},
 {u'_i

Now let's <tt>\$group</tt> by channels (which is now a single item) and create a new count field that sums up 1 for each document:

In [45]:
list(cells.aggregate([
        {
            '$unwind': '$channels'
        },
        {
            '$group': {
                '_id': '$channels',
                'count': {'$sum': 1}
            }
        }
    ]))

[{u'_id': u'I CAN', u'count': 1},
 {u'_id': u'I h', u'count': 3},
 {u'_id': u'I K,Ca', u'count': 2},
 {u'_id': u'I M', u'count': 1},
 {u'_id': u'I Potassium', u'count': 1},
 {u'_id': u'I p,q', u'count': 2},
 {u'_id': u'I L high threshold', u'count': 2},
 {u'_id': u'I T low threshold', u'count': 3},
 {u'_id': u'I K', u'count': 2},
 {u'_id': u'I Na,p', u'count': 2},
 {u'_id': u'I N', u'count': 3},
 {u'_id': u'I A', u'count': 3},
 {u'_id': u'I Na,t', u'count': 2}]

We can <tt>\$sort</tt> the channels by count. A basic principle is that it's better to let your database do as much of the data processing as possible instead of trying to transfer data to Python and processing it there.

In [49]:
list(cells.aggregate([
        {
            '$unwind': '$channels'
        },
        {
            '$group': {
                '_id': '$channels',
                'count': {'$sum': 1}
            }
        },
        {
            '$sort': {'count': -1}
        }
    ]))

[{u'_id': u'I h', u'count': 3},
 {u'_id': u'I T low threshold', u'count': 3},
 {u'_id': u'I N', u'count': 3},
 {u'_id': u'I A', u'count': 3},
 {u'_id': u'I K,Ca', u'count': 2},
 {u'_id': u'I p,q', u'count': 2},
 {u'_id': u'I L high threshold', u'count': 2},
 {u'_id': u'I K', u'count': 2},
 {u'_id': u'I Na,p', u'count': 2},
 {u'_id': u'I Na,t', u'count': 2},
 {u'_id': u'I CAN', u'count': 1},
 {u'_id': u'I M', u'count': 1},
 {u'_id': u'I Potassium', u'count': 1}]

The -1 in the <tt>\$sort</tt> command above means to sort in descending order. To sort in ascending order, replace the -1 with a 1.

Warning: MongoDB has a limit on the amount of memory it will use for a sort. If your query requires too much, a straightforward sort will fail, although there are work-arounds.

An even more flexible solution for data processing is to write your own <tt>map</tt> and <tt>reduce</tt> functions in JavaScript (yes, even though you're otherwise working in Python) and pass them to map_reduce. A simple example is <a href="http://api.mongodb.com/python/current/examples/aggregation.html#map-reduce">here</a>.

<h1>Deleting documents, etc</h1>

In [31]:
result = cells.delete_many({'name': {'$regex': 'Dentate'}})

In [32]:
result.deleted_count

1

There is also a <tt>delete_one</tt> method that deletes only one match.<br/><br/>To delete an entire collection, use, e.g.:

In [33]:
sciencedb.drop_collection('cells')

To delete an entire database, use <tt>drop_database</tt>:

In [96]:
mongodb.drop_database('sciencedb')